In [3]:
import pandas
import numpy as np
KDDTrain_20Percent= "./data/KDDTrain+_20Percent.txt"
KDDTest= "./data/KDDTest+.txt"
correspondence = "./data/testing_attack_types or all.txt"

In [5]:
dct = dict()
with open(correspondence, "r") as f:
    for line in f:
        line = line.split(' ')
        dct[line[0]] = line[1][:-1]
dct["normal"] = "normal" #最后一个字母会没掉，直接这样处理

In [37]:
data=pandas.read_csv(KDDTest,header=None)
data.shape

(22544, 43)

In [30]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [31]:
data[41].value_counts()
statistics = {"dos": 0, "u2r": 0, "probe": 0, "r2l": 0, "normal": 0}
labelling  = {"dos": 1, "u2r": 2, "probe": 3, "r2l": 4, "normal": 0}

for i in range(data.shape[0]):
    intrusion_class = dct[data.iat[i, 41]]
    data.iat[i, 41] = labelling[intrusion_class]
    statistics[intrusion_class] +=1 
    
statistics

{'dos': 9234, 'u2r': 11, 'probe': 2289, 'r2l': 209, 'normal': 13449}

# 预处理--字符串转换

In [32]:
#将相应的非数字类型转换为数字标识即符号型数据转化为数值型数据
def find_index(x,y):
    return [i for i in range(len(y)) if y[i]==x]
 
#定义将源文件行中3种协议类型转换成数字标识的函数
def handleProtocol(input):
    protocol_list=['tcp','udp','icmp']
    if input[1] in protocol_list:
        return find_index(input[1],protocol_list)[0]
    
#定义将源文件行中70种网络服务类型转换成数字标识的函数--测试集多一个'icmp'，加一个变71
def handleService(input):
    service_list=['aol','auth','bgp','courier','csnet_ns','ctf','daytime','discard','domain','domain_u',
                 'echo','eco_i','ecr_i','efs','exec','finger','ftp','ftp_data','gopher','harvest','hostnames',
                 'http','http_2784','http_443','http_8001','imap4','IRC','iso_tsap','klogin','kshell','ldap',
                 'link','login','mtp','name','netbios_dgm','netbios_ns','netbios_ssn','netstat','nnsp','nntp',
                 'ntp_u','other','pm_dump','pop_2','pop_3','printer','private','red_i','remote_job','rje','shell',
                 'smtp','sql_net','ssh','sunrpc','supdup','systat','telnet','tftp_u','tim_i','time','urh_i','urp_i',
                 'uucp','uucp_path','vmnet','whois','X11','Z39_50','icmp']
#     service_list=data[2].unique().tolist()
    if input[2] in service_list:
        return find_index(input[2],service_list)[0]
     
#定义将源文件行中11种网络连接状态转换成数字标识的函数
def handleFlag(input):
    flag_list=['OTH','REJ','RSTO','RSTOS0','RSTR','S0','S1','S2','S3','SF','SH']
    if input[3] in flag_list:
        return find_index(input[3],flag_list)[0]

#定义将源文件行中攻击类型转换成数字标识的函数(训练集中共出现了22个攻击类型，而剩下的17种只在测试集中出现)
def handleLabel(input):
    #label_list=['normal.', 'buffer_overflow.', 'loadmodule.', 'perl.', 'neptune.', 'smurf.',
    # 'guess_passwd.', 'pod.', 'teardrop.', 'portsweep.', 'ipsweep.', 'land.', 'ftp_write.',
    # 'back.', 'imap.', 'satan.', 'phf.', 'nmap.', 'multihop.', 'warezmaster.', 'warezclient.',
    # 'spy.', 'rootkit.']
    global label_list  #在函数内部使用全局变量并修改它
    if input[41] in label_list:
        return find_index(input[41],label_list)[0]
    else:
        label_list.append(input[41])
        return find_index(input[41],label_list)[0]
    
def handleLabel1(input):
    labelling  = {"dos": 1, "u2r": 2, "probe": 3, "r2l": 4, "normal": 0}
    intrusion_class = dct[input[41]]
    return labelling[intrusion_class]

In [38]:
#定义kdd99数据预处理函数
def preHandel_data():
    source_file=KDDTest
    handled_file='KDDTest.csv'
    data_file=open(handled_file,'w',newline='')     #python3.x中添加newline=''这一参数使写入的文件没有多余的空行
    with open(source_file,'r') as data_source:
        csv_reader=csv.reader(data_source)
        csv_writer=csv.writer(data_file)
        count=0   #记录数据的行数，初始化为0
        for row in csv_reader:
            temp_line=np.array(row)   #将每行数据存入temp_line数组里
            temp_line[1]=handleProtocol(row)   #将源文件行中3种协议类型转换成数字标识
            temp_line[2]=handleService(row)    #将源文件行中70种网络服务类型转换成数字标识
            temp_line[3]=handleFlag(row)       #将源文件行中11种网络连接状态转换成数字标识
            temp_line[41]=handleLabel1(row)   #将源文件行中23种攻击类型转换成数字标识
            csv_writer.writerow(temp_line)
            count+=1
            #输出每行数据中所修改后的状态
#             print(count,'status:',temp_line[1],temp_line[2],temp_line[3])
        data_file.close()

In [39]:
import time
import csv
start_time=time.clock()
global label_list   #声明一个全局变量的列表并初始化为空
label_list=[]
preHandel_data()
end_time=time.clock()
print("Running time:",(end_time-start_time))  #输出程序运行时间

Running time: 1.3609964999999988
